# Mesh and bones to rig network (MeshBonesToRigNet)

## Motivation

Designing a neural network for predicting skinning weights from a 3D mesh and its skeleton is an exciting challenge because it will help out the tedious, manual weight painting with an automated process that learns the intricate relationship between a character’s surface geometry and its underlying bone structure. By doing so, it can produce smoother, more natural deformations that adapt to a wide variety of poses and body types, ultimately enhancing realism in animations. Moreover, this endeavour bridges the gap between traditional computer graphics and modern machine learning techniques, offering a scalable solution that can handle diverse models and complex topologies with minimal human intervention. This blend of art and technology can accelerate production workflows and ease the burden on artists.

I got inspired to tackle this problem by my husband who is generally a programmer but also a 3D artist. He often needs to paint weights for his characters and never really finishes them (only enough so he can get them into the pose he wants). It's a very tedious task. So I thought it would be a fun project to automate it. It is ambitious to build something from scratch, but I think it is a great way to learn about neural networks and 3D geometry.
Well, I did find some papers which aided me like [RigNet](https://arxiv.org/abs/2005.00559) and later did I find after building the architecture a very recent paper which in some sense made me more confident in my approach [HumanRig](https://arxiv.org/abs/2412.02317).

I would have liked to have more time to work on this project before the deadline, but as always time was not on my side. I experimented more with how to build the architecture which meant that mlflow, tensorboard or optuna runs were not very meaningful and that is why I will layout here in this notebook my experiments and struggles until I made the network spit out something more meaningful than just favouring one bone for all vertices or uniform skinning for all bones. In the notebooks after number 3 I will use tensorboard and mlflow to record the experiments and hopefully I have time to experiment with hyperparameter adjustments and run a few experiments using optuna (fingers crossed). You can't imagine how much time it took to adjust the architecture, think of more meaningful loss functiona and preprosses the data (I took the RigNet dataset and was glad they also had a preproccessed version (details on what it coins later) but it was not exactly what I needed, so I had to preprocess it myself). I will begin with the current state of the architecture and then go back to talk about how the modules evolved.

## Overview of the final architecture

                                                                                    +--------------------------------+
                                                                                    |         Input Module           |
                                                                                    |  - Mesh: vertices, faces, etc  |
                                                                                    |  - Bone Structure (graph)      |
                                                                                    +----------------+---------------+
                                                                                    │                                 |
                                                                                    ▼                                 ▼
                                                +------------------------------------------+                    +------------------------------------------+                                                     
        Vertex features H_v ∈ ℝ^(N x d_v)  <--  |            Mesh Encoder                  |                   |         Bone Encoder                      | --> Bone embeddings H_b ∈ ℝ^(B x d_b)       
                                                +------------------------------------------+                    +------------------------------------------+                                                      
                                                                                    │                               │
                                                                                    ▼                               ▼
                                                                            +------------------------------------------+
                                                                            |               Fusion Module               | --> Predicted skinning weights W_pred ∈ ℝ^(N x B)
                                                                            |       (Attention: compute similarity      |
                                                                            |     between each vertex and each bone)    |
                                                                            +------------------------------------------+
                                                                                                    │
                                                                                                    ▼
                                                                            +------------------------------------------+
                                                                            |         Refinement Module                | --> Refined skinning weights W_final ∈ ℝ^(N x B)
                                                                            |           (Optional for now,             |
                                                                            |        needs more experimenting)         |
                                                                            +------------------------------------------+
                                                                                                │
                                                                                                ▼
                                                                            +------------------------------------------+
                                                                            |        Output: Skin Weights              |
                                                                            +------------------------------------------+

**MeshBonesToRigNet**

* **Inputs:**
  * **Vertices:** (N, 3) tensor of vertex positions.
  * **Edge Index Geodesic:** (2, N*k) LongTensor precomputed from the surface geodesic distance matrix.
  * **Edge Attr Geodesic:** (N*k, 1) tensor precomputed from the surface geodesic distance matrix.
  * **Vertex Neighbors:** Precomputed indices (N, k) derived from the surface geodesic distance matrix.
  * **Vertex Adjacency:** Dense, normalized (N, N) matrix from face connectivity.
  * **Vertex Normals:** (N, 3) tensor.
  * **Bone Features:** (B, 8) tensor (joint positions and additional features).
  * **Bone Adjacency:** Normalized (B, B) matrix from the rig’s hierarchy.
  * **Volumetric Geodesic:** (N, B) tensor (volumetric distances from vertices to bones).
  * **Surface Geodesic:** Full (N, N) surface geodesic matrix.
N is the number of vertices and B is the number of bones, and k is the number of the closest neighbors for each vertex to be considered.

* **Modules:**
  1. **Mesh Encoder (SplineConv-based):**  
     Processes vertex positions using a graph constructed from the geodesic-based k‑NN. The SplineConv layers leverage continuous edge attributes (e.g. geodesic distance) to capture local geometry.
  2. **Bone Encoder (GCNConv-based):**  
     Processes enriched bone features (which include joint positions and possibly bone lengths/directions) along the skeletal hierarchy.
  3. **Fusion Module (Sparse Attention with Volumetric Bias):**  
     Fuses per-vertex mesh features with bone embeddings, using volumetric geodesic distances w.r.t. vertices and bones to bias attention scores and produce an initial skinning weight distribution.
  4. **Refinement Module:**  
     The idea is to smooth the initial predictions (for example, via diffusion or adaptive gating) to ensure local consistency in regions with positive influence. For now I have not been able to come to a final version of this module.

Now some explanaition and history of the modules:


## What I had gone through to get to that architecture

### Choices for Mesh Encoder and BoneEncoder base layers

I initially wanted to integrate a layer from a paper I reaserched [Neural Mesh Simplification](https://ieeexplore.ieee.org/document/9878835) called **DevConv** which promised to aggregate information based on the relative coordinates of neighbouring points, enhancing the model's ability to identify topological features and detect outliers. But as I was experimenting later with the architecture, I found out that I got zero gradient updates for the DevConv layer. I have not figured out why this is the case. I tried different initialization schemes (I tried the default one and Xavier uniform initialization), but it didn't help. With time running out I reaserched  the next best thing to **GCNConv** (used in the Bone Endoder from the beginning) and that was **SplineConv**. **SplineConv**, with its use of continuous B‑spline basis functions, naturally integrates continuous geometric cues, which not only better reflects the irregular geometry of meshes but also results in more stable and meaningful gradient updates which was what I needed at the time. There wwere other choises like:
* **GCNConv** - a simpel graph convolution that aggregates neighboring features by averaging. It is less expressive because it typically does not integrate continuous edge attributes. That is why I though it was a good fit for the bone encoder as the skleton is really a graph tree where joints are nodes connected by clear parent–child relationships.
* **EdgeConv** - Often used in dynamic graph CNNs (DGCNN), it computes local edge features and is popular in point cloud processing.
* **ChebConv** - A spectral approach that approximates convolutions via Chebyshev polynomials. It can capture local structure well.

In the beginning I wasn't using any additional features for the mesh and bone encoder apart from their adjacency matrices which yielded poor results and only when I encorporated surface geodesic distance based features for the mesh encoder and bone length and direction for the bone encoder the results improved.

I didn't want the architecture of the modules to be too complex so the modules are not too deep with not too many parameters (they are customizable though). There are no skip connections of any type or pooling/unpooling layers which may be interesting to try out in a later stage. They both output a feature vector for each vertex and bone respectively to the fusion module.

### Evolution of the Fusion Module

It started out as a very, very simple computation of attention scores between each vertex and each bone while first projecting their features to a common dimension followed by a softmax operation. After some reaserch I found out that on average a vertex might be influenced from up to 4 bones which gave me the idea to incorporate top-k highest scores for each vertex to form a sparse attention distribution, which in turn was normalized via softmax. The idea behind using sparse attention in the fusion module is to force the network to consider only the most relevant bone influences for each vertex, rather than all bones. By restricting the attention computation to the top few bones, I reduced the noise from less relevant bones (in theory) and the network was able to learn the influence of bones on a vertex more accurately.

Yeah, but something was missing as the network was still favouring one bone for all vertices. That is when my experience for the last few years working with rendering and 3d geometry paid off (as well as all the other tasks I explored while doing projects for the AI module like mesh simplification, mesh segmentation, etc.) and I encorporated pre-calculated volumetric geodesic distances between each vertex and bone with exponential decay (ensures that bones with smaller geodesic distances,i.e., closer in the intrinsic geometry of the mesh are favored) to bias the attention (at the same time I encorporated the surface geodesic distance based features for the mesh encoder). That started to yield some meaningful results even tough it was trained only on one mesh and one rig for 1000 epochs.

### Why the Refinement Module is not used

Well, I wanted to create a module that would smooth the initial predictions and make them more consistent in regions with positive influence. I thought about something like this:
It refines per-vertex skin weight predictions using local neighborhood information weighted by vertex normal similarity. For each vertex $i$, let $p_i$ be its predicted skinning weight vector (shape: $(B,)$), and let $N(i)$ be the set of $k$-nearest neighbors (provided by vertex_neighbors, shape $(k,)$). For each neighbor $j$ in $N(i)$, we compute a similarity score:
\begin{equation*}
        s_{ij} = max(0, \dot(n_i, n_j)),
\end{equation*}
where $n_i$ and $n_j$ are the normalized vertex normals. We then compute a diffused version:    
\begin{equation*}
        \text{diffused}_i = (\sum_{j \in N(i)} s_{ij} * p_j) / (\sum_{j \in N(i)} s_{ij} + eps)
\end{equation*}
Finally, we let the refined logits be a weighted combination of the original prediction and the diffused one:
\begin{equation*}
        \text{refined logits}_i = p_i + \gamma * (\text{diffused}_i - p_i)
\end{equation*}
and apply softmax to obtain the refined skinning weight distribution.
The parameter gamma ($\gamma$) is a learnable scalar.

Well it might sound reasonable, but for now when I apply it the loss functions seem to converge better, but the visual results are worse. I will have to experiment more with it.

I mentioned the loss functions so might as well transition to them.

### Loss functions

I started with a simple loss function that computed the mean squared error between the predicted skinning weights and the ground truth skinning weights. But it yielded poor results. So I tried a few other loss functions:

##### 1. Skinning Weight Loss (KL Divergence Loss)

**Purpose:**  
The Kullback-Leibler (KL) divergence loss measures the difference between two probability distributions. In the context of skinning weights, it would measure how different the predicted weight distribution is from the target distribution.

**Description:**  
For each vertex $i$, let:
- $p_i^{\text{pred}}$ be the predicted skinning weight vector (of length $B$).
- $p_i^{\text{target}}$ be the ground-truth (target) skinning weight vector.

The loss is computed as the average KL divergence across vertices:
\begin{equation*}
\mathcal{L}_{\text{skin}} = \frac{1}{N} \sum_{i=1}^{N} \mathrm{KL}\left(p_i^{\text{target}} \,\|\, p_i^{\text{pred}}\right)
\end{equation*}
for
\begin{equation*}
\mathrm{KL}\left(p_i^{\text{target}} \,\|\, p_i^{\text{pred}}\right) = \sum_{j=1}^{B} p_i^{\text{target}}[j] \log \left(\frac{p_i^{\text{target}}[j]}{p_i^{\text{pred}}[j]}\right)
\end{equation*}

**Intuition:**  
Minimizing this loss encourages the network to output distributions that closely match the rig’s target skin weights for each vertex.

##### 2. Deformation Consistency Loss (Optional)

**Purpose:**  
To ensure that the mesh deforms similarly when using the predicted skin weights as it does with the target skin weights.

**Description:**  
Assume we have a differentiable skinning function (e.g., Linear Blend Skinning) that computes deformed vertex positions. Let:
- $\hat{x}_i^{\text{pred}}$ be the deformed position using the predicted skin weights.
- $\hat{x}_i^{\text{target}}$ be the deformed position using the target skin weights.

The loss is defined as:
\begin{equation*}
\mathcal{L}_{\text{deform}} = \frac{1}{N} \sum_{i=1}^{N} \left\| \hat{x}_i^{\text{pred}} - \hat{x}_i^{\text{target}} \right\|^2
\end{equation*}

**Intuition:**  
This loss helps ensure that even if the predicted weights are not exactly the same as the target weights, the resulting deformation of the mesh is still consistent with the desired outcome. The problem was that I hadn't the time to compute different deformations which means that this loss is a good to have thing for future experiments.

##### 3. Smoothness Loss

**Purpose:**  
To promote local consistency in the predicted skinning weights across the mesh.

**Description:**  
For each vertex $i$, consider its set of neighbors $\mathcal{N}(i)$ (defined via the vertex adjacency matrix). The loss penalizes large differences in skinning weights between vertex $i$ and each neighbor $j$:
\begin{equation*}
\mathcal{L}_{\text{smooth}} = \frac{1}{N} \sum_{i=1}^{N} \sum_{j \in \mathcal{N}(i)} \left\| p_i^{\text{pred}} - p_j^{\text{pred}} \right\|^2
\end{equation*}

**Intuition:**  
This loss encourages neighboring vertices to have similar skinning weight distributions, which is particularly important when vertices have multiple nonzero weights. The Laplacian or neighbor-averaging regularization does not assume a single dominant influence; rather, it works with the entire weight distribution, promoting smooth transitions even when several bones influence a vertex.

##### 4. Geodesic Loss (Volumetric Geodesic Bias Loss)

**Purpose:**  
To incorporate intrinsic geometric information into the predictions so that vertices closer (in the intrinsic sense) to a bone receive higher influence.

**Description:**  
For each vertex $i$ and bone $j$, let $d_{ij}$ be the volumetric geodesic distance. We compute a geometry-based target distribution using an exponential decay:
\begin{equation*}
p_{ij}^{\text{geo-target}} = \frac{\exp(-\alpha\, d_{ij})}{\sum_{k=1}^{B} \exp(-\alpha\, d_{ik})}
\end{equation*}
Then, the geodesic loss is defined as:
\begin{equation*}
\mathcal{L}_{\text{geo}} = \frac{1}{N} \sum_{i=1}^{N} \mathrm{KL}\left(p_i^{\text{geo-target}} \,\|\, p_i^{\text{pred}}\right)
\end{equation*}

**Intuition:**  
The exponential decay ensures that bones with smaller geodesic distances (i.e., closer in the intrinsic geometry of the mesh) are favored. Minimizing this loss encourages the network to assign higher weights to nearby bones, aligning the predictions with the underlying mesh geometry.

##### 5. Overall Loss

The total loss is a weighted combination of the above components:
\begin{equation*}
\mathcal{L}_{\text{total}} = \lambda_{\text{skin}}\, \mathcal{L}_{\text{skin}} \, + \lambda_{\text{smooth}}\, \mathcal{L}_{\text{smooth}} + \lambda_{\text{geo}}\, \mathcal{L}_{\text{geo}}
\end{equation*}

- **$\lambda$ Parameters:**  
  These hyperparameters control the relative importance of each loss component.
  
**Intuition of the Combined Loss:**  
The overall is there to ensure that:
- The skinning weight distributions match the target data (via $\mathcal{L}_{\text{skin}}$).
- Predictions vary smoothly across neighboring vertices (via $\mathcal{L}_{\text{smooth}}$).
- The influence of each bone respects the intrinsic geometry of the mesh (via $\mathcal{L}_{\text{geo}}$).

For now I get a mixed comresult from using this coombined functions with all the lambdas set to 1. The Skin loss gest lower, while the Geo loss gets higher (at least with the training on one mesh for 1000 epochs). I will have to experiment more to say how effective is this combination, how should I weight them, etc.

### Dataset gathering and preprocessing


A lot of my time was speant lookig for a dataset for the task until I found the [RigNet](https://arxiv.org/abs/2005.00559) paper. From the [github repo](https://github.com/zhan-xu/RigNet) I found links to the dataset they used and a preprocessed version of it:
* https://drive.google.com/file/d/1yojBwl5eHPqgXZ1Uh4j26S-yKK-2loPu/view;
* https://drive.google.com/file/d/1-B6hJ4423rw1LrTForHp7oaG5qRAbJx3/view.

The ModelsResource-RigNetv1 has 2,703 models split into 80% for training (2,163 models), 10% for validation (270 models), and 10% for testing. All models in fbx format in the original dataset. They have shared a preprocessed version of it which consists of several subfolders ([cited from](https://github.com/zhan-xu/RigNet#:~:text=several%20sub%2Dfolders,grid%20is%2088x88x88.)):
* obj: all meshes in OBJ format.
* rig_info: we store the rigging information into a txt file. Each txt file has four blocks. (1) Lines starting with "joint" define a joint with its 3D position. Each of joint line has four elements, which are joint_name, X, Y, and Z. (2) Line starting with "root" defines the name of root joint. (3) Lines starting with "hier" define the hierarchy of skeleton. Each hierarchy line has two elements, which are parent joint name and its child joint name. One parent joint can have multiple children joints. (4) Lines starting with "skin" define the skinning weights. Each skinning line follows the format as vertex_id, bind_joint_name_1, bind_weight_1, bind_joint_name_2, bind_weight_2 ... The vertex_id follows the vertice order in obj files in the above obj folder.
* obj_remesh: This folder contains the obj files of the remeshed models. Meshes with fewer than 1K vertices were subdivided, and those with more than 5K vertices were simplified; as a result all training and test meshes contained between 1K and 5K vertices.
* rig_info_remesh: Rigging information files corresponding to the remeshed obj. Joints, hierarchy and root are the same. The skinning is recalculated based on nearest neighbor from each remeshed vertex to original vertices.
* pretrain_attention: Pre-calculated supervision to pretrin the attention module, which are calculated by the script geometric_proc/compute_pretrain_attn.py. Each file is a N-by-1 text where N is the number of vertices corresponding to remeshed OBJ file, the i-th row stores the surpervision for vertex i.
* volumetric_geodesic: Pre-calculated volumetric geodesic distance between each vertex-bones pair. The algorithm is an approaximation, which is implemented in geometric_proc/compute_volumetric_geodesic.py. Each file is an N-by-B numpy array where N is the number of vertices corresponding to remeshed OBJ file, B is the number of bones, and (i, j) stores the volumetric geodesic distance between vertex i and bone j.
* vox: voxelized models used for inside/outside check. Obtained with binvox. The resolution of the grid is 88x88x88.

I was very happy to use obj_remesh and rig_info_remesh and I wanted to use volumetric_geodesic, but after a bit of reaserach I found out that the bone dimension of the matrix was more than the bone number recorder in the rig information for the mesh. That was due to the fact that they compurted the metric for virtual "leaf" bones which are zero-length and are added to ensure that leaf joints, which often have skinning weights, are appropriately represented. For instance, if a skeleton has 10 actual bones and 5 leaf joints, then we will have a total of 15 bones (10 actual + 5 virtual leaf bones). Consequently, the .npy file will have a second dimension size corresponding to these 15 bones. This design ensures that all joints, including leaf joints, are considered during processes like skinning, where each joint's influence on the mesh deformation is important. But as I started to try to integrate their work for my needs I came to realize that it would take more time to do that rather than just write a really naive and potentially slower implementation myself. That is why I chose to do it myself. In the 1st notebook there is a cell which will precompute the volumentric geodesic distances and surface geodesic distances (verrry slow) for the whole dataset. I will add a gdrive link to that data if one needs it (it is ~100GB and that is mainly due to the $N\times N$ matrices for the surface geodesic distances). In practice here I will use a dataset of say 118 meshes which is much more fesable if I have the time to do some more serious training (100-9-9 train-val-test split).

One of my biggest problems is still that I cannot train with batch size greater than 1. When batching graphs with variable numbers of vertices, torch‑geometric’s default collate (Batch.from_data_list) can handle variable‑sized node features (like vertices and edge_index) because it concatenates node features and shifts indices accordingly. However, attributes that are "square" (like vertex adjacency matrices or full surface geodesic matrices of shape $(N, N)$) don’t batch naturally because each graph may have a different $N$. It is one of my todos, I want to be able to train with batch size at least 2 (which is a reasonable number considering how much stuff I have put in a single Data object).

### What I will do now with the short time I have

I will try to experiment and document with tensorboard different scenarios like with or without refinement, play with the lambdas of the loss functions, etc. If I have the time I would like to train on at least 100 meshes to see how the network performs on unseen data (I will use the train-val-test split of 100-9-9).